In [17]:
from finn.util.visualization import showInNetron
showInNetron("./citrinet-finn-hw.onnx")

Stopping http://0.0.0.0:8081
Serving './citrinet-finn-hw.onnx' at http://0.0.0.0:8081


In [1]:
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

model = ModelWrapper("./citrinet.onnx")
model = model.transform(ConvertQONNXtoFINN())
model.save("./citrinet-finn.onnx")

/home/npawolka/finn/src/finn/transformation/qonnx/qonnx_activation_handlers.py:412: UserWarning: No layout annotations for Quant_279_out0: Assuming channel dimension at index 1
  warnings.warn(
/home/npawolka/finn/src/finn/transformation/qonnx/qonnx_activation_handlers.py:412: UserWarning: No layout annotations for Quant_281_out0: Assuming channel dimension at index 1
  warnings.warn(
/home/npawolka/finn/src/finn/transformation/qonnx/qonnx_activation_handlers.py:412: UserWarning: No layout annotations for Quant_284_out0: Assuming channel dimension at index 1
  warnings.warn(
/home/npawolka/finn/src/finn/transformation/qonnx/qonnx_activation_handlers.py:412: UserWarning: No layout annotations for Quant_286_out0: Assuming channel dimension at index 1
  warnings.warn(
/home/npawolka/finn/src/finn/transformation/qonnx/qonnx_activation_handlers.py:412: UserWarning: No layout annotations for Quant_288_out0: Assuming channel dimension at index 1
  warnings.warn(
/home/npawolka/finn/src/finn/t

In [3]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants

model = ModelWrapper("./citrinet-finn.onnx")
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save("./citrinet-finn-tidy-up.onnx")

In [5]:
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.streamline import Streamline
import finn.transformation.streamline.absorb as absorb

model = ModelWrapper("./citrinet-finn-tidy-up.onnx")
model = model.transform(absorb.AbsorbSignBiasIntoMultiThreshold())
model = model.transform(Streamline())
model.save("./citrinet-finn-streamlined-pre_lower.onnx")


/home/npawolka/finn/deps/qonnx/src/qonnx/core/modelwrapper.py:382: UserWarning: find_consumer: found multiple consumers, returning first one
  warnings.warn("find_consumer: found multiple consumers, returning first one")


In [14]:
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
import finn.transformation.streamline.absorb as absorb
import finn.transformation.streamline.reorder as reorder
from finn.transformation.streamline.transpose import CollapseRepeatedTranspose
from qonnx.transformation.infer_data_layouts import InferDataLayouts
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds
from finn.transformation.streamline.collapse_repeated import CollapseRepeatedMul


from qonnx.core.modelwrapper import ModelWrapper

model = ModelWrapper("./citrinet-finn-streamlined-pre_lower.onnx")

model = model.transform(reorder.MoveScalarLinearPastInvariants())
model = model.transform(reorder.MoveLinearPastEltwiseAdd())
model = model.transform(reorder.MoveAddPastFork())
model = model.transform(reorder.MoveMulPastFork())
model = model.transform(absorb.AbsorbAddIntoMultiThreshold())
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())

model = model.transform(LowerConvsToMatMul())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(reorder.MoveTransposePastFork())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(InferDataLayouts())
model = model.transform(reorder.MoveTransposePastMul())
model = model.transform(reorder.MoveTransposePastJoinAdd())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(absorb.AbsorbConsecutiveTransposes())

model = model.transform(to_hw.InferGlobalAccPoolLayer())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(absorb.AbsorbConsecutiveTransposes())
model = model.transform(absorb.AbsorbMulIntoMultiThreshold())

model = model.transform(reorder.MoveTransposePastAdd())
model = model.transform(reorder.MoveTransposePastMul())
model = model.transform(CollapseRepeatedTranspose())
model = model.transform(reorder.MoveAddPastMul())
model = model.transform(CollapseRepeatedMul())

#model = model.transform(reorder.MoveMultiThresholdPastTranspose())
model = model.transform(RoundAndClipThresholds())
model = model.transform(reorder.MoveMultiThresholdBeforeFork())
model = model.transform(reorder.MoveMatMulPastFork())

model.save("./test-lower.onnx")

In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from convert_to_hw import InferQuantizedMatrixVectorActivation

model = ModelWrapper("./test-lower.onnx")

model = model.transform(to_hw.InferElementwiseBinaryOperation())
#model = model.transform(to_hw.InferBinaryMatrixVectorActivation())
model = model.transform(to_hw.InferVectorVectorActivation())
# needed for non-bipolar MatMul layers
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())

model.save("./citrinet-vector.onnx")

# TopK to LabelSelect
#model = model.transform(to_hw.InferLabelSelectLayer())
# input quantization (if any) as standalone threshold
model = model.transform(to_hw.InferThresholdingLayer())
# needed for convolutions -- TODO always exec?
need_conv = len(model.get_nodes_by_op_type("Im2Col")) > 0
if need_conv:
    model = model.transform(to_hw.InferConvInpGen())
    #model = model.transform(to_hw.InferStreamingMaxPool())
    #model = model.transform(RemoveCNVtoFCFlatten())
# get rid of Tranpose -> Tranpose identity seq
#model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
#model = model.transform(absorb.AbsorbConsecutiveTransposes())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(InferDataLayouts())

model = model.transform(to_hw.InferDuplicateStreamsLayer())

model.save("./citrinet-finn-hw.onnx")

In [ ]:
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.infer_data_layouts import InferDataLayouts
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
import finn.transformation.streamline.absorb as absorb
import finn.transformation.streamline.reorder as reorder
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds

from qonnx.transformation.infer_datatypes import InferDataTypes

from finn.transformation.streamline.transpose import CollapseRepeatedTranspose

from qonnx.core.modelwrapper import ModelWrapper

model = ModelWrapper("./citrinet-finn-hw.onnx")

model = model.transform(to_hw.InferDuplicateStreamsLayer())

model.save("./test.onnx")

In [18]:
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as oxe
import torch

model = ModelWrapper("./citrinet.onnx")
input_tensor = torch.randn((1, 80, 8192))
input_dict = {"global_in": input_tensor.cpu().numpy() }
output_dict = oxe.execute_onnx(model, input_dict)
produced_qonnx = output_dict[list(output_dict.keys())[0]]

In [61]:
input_tensor.unsqueeze(-1).shape

torch.Size([1, 80, 8192, 1])

In [19]:
model = ModelWrapper("./citrinet-finn-hw.onnx")
input_dict = {"global_in": input_tensor.cpu().numpy() }
output_dict = oxe.execute_onnx(model, input_dict)
produced_finn = output_dict[list(output_dict.keys())[0]]

/home/npawolka/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor eK0ofg can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/npawolka/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor hcDTUe can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/npawolka/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor cR2huG can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/npawolka/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor 2lqx10 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
/home/npawolka/finn/deps/qonnx/src/qonnx/util/basic.py:296: 

In [20]:
torch.allclose(torch.from_numpy(produced_qonnx).squeeze(), torch.from_numpy(produced_finn).squeeze(), rtol=0.01, atol=100 * 0.01)

True

In [22]:
produced_finn

array([[[-5.5490794, -5.549089 , -5.5490694, ..., -5.5490756,
         -5.5490785, -5.5490785],
        [-5.5490794, -5.549089 , -5.5490694, ..., -5.5490756,
         -5.5490785, -5.5490785],
        [-5.5490794, -5.549089 , -5.5490694, ..., -5.5490756,
         -5.5490785, -5.5490785],
        ...,
        [-5.5490794, -5.549089 , -5.5490694, ..., -5.5490756,
         -5.5490785, -5.5490785],
        [-5.5490794, -5.549089 , -5.5490694, ..., -5.5490756,
         -5.5490785, -5.5490785],
        [-5.5490794, -5.549089 , -5.5490694, ..., -5.5490756,
         -5.5490785, -5.5490785]]], dtype=float32)